In [1]:
import pandas as pd

In [2]:
from konlpy.tag import Okt

In [3]:
okt = Okt()

In [2]:
file_path = 'SentiWord_Dict.txt'

In [3]:
# 사전 데이터 불러오기
df = pd.read_csv('SentiWord_Dict.txt', delimiter='\t', names=['단어', '점수'], header=None)

In [4]:
# 한글 부분만 필요하기 때문에, 사전에서 한글필터링 - 정규표현식
df = df[df['단어'].str.match(r'^[가-힣]+$')]

In [5]:
df.head(10)

,단어,점수
44,가까스로,0.0
46,가까이하다,1.0
47,가꾸러뜨리다,-1.0
48,가꾸러트리다,-1.0
49,가난,-2.0
50,가난뱅이,-2.0
51,가난살이,-2.0
52,가난살이하다,-2.0
53,가난설음,-2.0
54,가난에,-2.0


In [23]:
positive_words= ['상승','호재','수익','매수','강세',
                 '성장','기대감','안정성','배당','흥행',
                 '신규','투자','우량주','전망','실적',
                 '신뢰','회복','대박','상한가','수익률',
                 '투더문','떡상','상승세','신고가','역대급',
                 '황금주','대세','상승','급등','발전',
                 '전환점','인수합병','신제품','신기술','비전',
                 '선도','전망치','잭팟','우상향','강세장','화성']

In [27]:
positive_df = pd.DataFrame({'단어': positive_words, '점수': [1.0] * len(positive_words)})

In [31]:
positive_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   단어      41 non-null     object 
 1   점수      41 non-null     float64
dtypes: float64(1), object(1)
memory usage: 788.0+ bytes


In [4]:
negative_words = ['손절','폭락','물렸다','하한가','떡락',
                  '개미털기','작전주','버블','폭탄주','축소',
                  '물타기','매도','공매도','붕괴','실패','하락',
                  '피눈물','악재','손해','물로켓','부도','리콜',
                  '변동성','퇴출','파산','잠식','위기','침체',
                  '불황','나락','폭탄','이탈','손실','냉각',
                  '마이너스','폭망','미달','부진','악화','적자',
                  '위험','불확실','불안정','혼란','실망','부정적',
                  '악성','비관적','걱정','우려','패닉','실의',
                  '고통','포기','탈출','회피','불만','퇴보',
                  '사라지다','문제'
                  ]

In [5]:
negative_df = pd.DataFrame({'단어': negative_words, '점수': [-2.0] * len(negative_words)})

In [29]:
df = pd.concat([df, positive_df], ignore_index=True)

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6425 entries, 0 to 6424
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   단어      6425 non-null   object 
 1   점수      6425 non-null   float64
dtypes: float64(1), object(1)
memory usage: 100.5+ KB


In [6]:
df = pd.read_csv('positive_score_dict.csv')

In [8]:
df = pd.concat([df, negative_df], ignore_index=True)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6435 entries, 0 to 6495
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   단어      6435 non-null   object 
 1   점수      6435 non-null   float64
dtypes: float64(1), object(1)
memory usage: 150.8+ KB


In [12]:
df.to_csv('positive_score_dict.csv', index=False, encoding='utf-8-sig')

In [37]:
df.head(5)

,단어,점수
0,가까스로,0.0
1,가까이하다,1.0
2,가꾸러뜨리다,-1.0
3,가꾸러트리다,-1.0
4,가난,-2.0


In [39]:
df = df[df['단어'].str.len() > 1]

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6436 entries, 0 to 6461
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   단어      6436 non-null   object 
 1   점수      6436 non-null   float64
dtypes: float64(1), object(1)
memory usage: 150.8+ KB


In [41]:
df.to_csv('positive_score_dict.csv', index=False, encoding='utf-8-sig')

In [16]:
# 긍정 점수 사전 읽기
positive_dict = pd.read_csv('positive_score_dict.csv')
positive_dict = positive_dict.drop_duplicates(subset='단어')
positive_dict.set_index('단어', inplace=True)

In [17]:
positive_dict.head()

,점수
단어,
가까스로,0.0
가까이하다,1.0
가꾸러뜨리다,-1.0
가꾸러트리다,-1.0
가난,-2.0


In [19]:
community_contents = pd.read_csv('community_token_final.csv', low_memory=False)

In [14]:
community_contents.describe()

,닉네임,날짜,내용,종목,내용_명사,내용_형용사,내용_부사,내용_동사,내용_명사_str,내용_형용사_str,내용_부사_str,내용_동사_str
count,81072,81072,81072,81072,81072,81072,81072,81072,71966,25853,13255,55231
unique,24593,75710,74750,537,55032,12190,1955,36423,55031,12189,1954,36422
top,특징주알리미,2025-01-02T10:22:42+09:00,.,5930,[],[],[],[],가즈,입니다,다,가자
freq,306,5,260,10263,9106,55219,67817,25841,304,409,2172,649


In [ ]:
# 행별 총 긍정 점수를 계산할 파생열 추가
row_scores = [0] * len(community_contents)  # 초기화

for idx in range(len(community_contents)):  # 데이터프레임 행 반복
    row_score = 0  # 행별 총 점수 초기화

    for column in ['내용_명사_str', '내용_형용사_str', '내용_부사_str', '내용_동사_str']:
        word_list = community_contents.loc[idx, column]  # 각 열의 데이터 가져오기
        print(f"\n행 {idx}, 열 {column} 처리 중: {word_list}")
        
        if pd.isna(word_list) or word_list.strip() == "":  # Null 또는 빈 문자열 처리
            print(f"행 {idx}, 열 {column}은 결측값 또는 빈 문자열입니다. 건너뜁니다.")
            continue
        
        words = word_list.split(', ')  # 쉼표와 공백으로 구분
        for word in words:
            word = word.strip()  # 공백 제거
            if word in positive_dict.index:
                score = positive_dict.loc[word, '점수']
                row_score += score  # 점수 합산
                print(f"  단어: {word}, 점수: {score}, 누적 점수: {row_score}")
            else:
                print(f"  단어: {word}는 사전에 없음")

    row_scores[idx] = row_score  # 행별 총 점수 저장
    print(f"행 {idx}의 총 점수: {row_score}")

# 점수 변환: 음수는 -1, 0은 0, 양수는 1
try:
    community_contents['긍정점수'] = [1 if score > 0 else -1 if score < 0 else 0 for score in row_scores]

except Exception as e:
    print(f"점수 변환 중 에러 발생: {e}")
    print(f"row_scores 내용: {row_scores}")
    print(f"tmp 길이: {len(community_contents)}, row_scores 길이: {len(row_scores)}")

# 최종 점수 출력
print("\n행별 긍정점수:")
print(community_contents[['긍정점수']])

In [21]:
community_contents.head()

,닉네임,날짜,내용,종목,내용_명사,내용_형용사,내용_부사,내용_동사,내용_명사_str,내용_형용사_str,내용_부사_str,내용_동사_str,긍정점수
0,GROK,2025-01-11T22:59:18+09:00,- 디자인 변화: S25 울트라는 기존의 각진 디자인에서 둥근 모서리로 변경되었으며...,5930,"['디자인', '변화', '울트라', '기존', '진', '디자인', '모서리', ...","['둥근', '있습니다', '동일한', '있음']",[],"['되었으며', '와', '보입니다']","디자인, 변화, 울트라, 기존, 진, 디자인, 모서리, 변경, 카메라, 모듈, 디자...","둥근, 있습니다, 동일한, 있음",NaN,"되었으며, 와, 보입니다",1
1,공감아,2025-01-11T22:47:22+09:00,7만원 8만원에 물려있는게 대부분이라 7만원까지 혹여 올라도 물려있던분들은 원금 회...,5930,"['대부분', '혹', '던', '원금', '회복', '시', '다시', '풀', ...",[],[],"['물려있는게', '올라도', '물려있']","대부분, 혹, 던, 원금, 회복, 시, 다시, 풀, 매도, 다시",NaN,NaN,"물려있는게, 올라도, 물려있",0
2,곰돌이와콩이,2025-01-11T22:41:04+09:00,분할매수하기에 좋은 타점이지만.. 얘는 너무느리고 오래걸린다 얘를살바에 살게 너무많...,5930,"['분할', '타점', '얘', '얘', '살바', '살', '단점']","['좋은', '느리고', '많다는']","['너무', '오래', '너무']","['매수하기에', '걸린다']","분할, 타점, 얘, 얘, 살바, 살, 단점","좋은, 느리고, 많다는","너무, 오래, 너무","매수하기에, 걸린다",1
3,김브래,2025-01-11T21:31:55+09:00,존버하면 오르겠죠?,5930,['존버하'],[],[],['오르겠죠'],존버하,NaN,NaN,오르겠죠,0
4,김보리씨,2025-01-11T21:07:54+09:00,오를 겁니다. 제가 오늘 열심히 일하고 퇴근했습니다.,5930,"['제', '오늘', '일', '퇴근']",[],['열심히'],"['오를', '겁니다', '했습니다']","제, 오늘, 일, 퇴근",NaN,열심히,"오를, 겁니다, 했습니다",1


In [23]:
community_contents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81072 entries, 0 to 81071
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   닉네임         81072 non-null  object
 1   날짜          81072 non-null  object
 2   내용          81072 non-null  object
 3   종목          81072 non-null  object
 4   내용_명사       81072 non-null  object
 5   내용_형용사      81072 non-null  object
 6   내용_부사       81072 non-null  object
 7   내용_동사       81072 non-null  object
 8   내용_명사_str   71966 non-null  object
 9   내용_형용사_str  25853 non-null  object
 10  내용_부사_str   13255 non-null  object
 11  내용_동사_str   55231 non-null  object
 12  긍정점수        81072 non-null  int64 
dtypes: int64(1), object(12)
memory usage: 8.0+ MB


In [25]:
community_contents['긍정점수'].value_counts()

긍정점수
 0    63025
 1    10524
-1     7523
Name: count, dtype: int64

In [ ]:
tmp.head(10)

In [ ]:
community_contents

In [29]:
# 각 열에 대해 okt.morphs(stem=True) 적용
community_contents['형용사_어간'] = community_contents['내용_형용사_str'].apply(lambda x: okt.morphs(x, stem=True) if isinstance(x, str) and x != 'NaN' else [])
community_contents['부사_어간'] = community_contents['내용_부사_str'].apply(lambda x: okt.morphs(x, stem=True) if isinstance(x, str) and x != 'NaN' else [])
community_contents['동사_어간'] = community_contents['내용_동사_str'].apply(lambda x: okt.morphs(x, stem=True) if isinstance(x, str) and x != 'NaN' else [])

In [30]:
community_contents.head()

,닉네임,날짜,내용,종목,내용_명사,내용_형용사,내용_부사,내용_동사,내용_명사_str,내용_형용사_str,내용_부사_str,내용_동사_str,긍정점수,형용사_어간,부사_어간,동사_어간
0,GROK,2025-01-11T22:59:18+09:00,- 디자인 변화: S25 울트라는 기존의 각진 디자인에서 둥근 모서리로 변경되었으며...,5930,"['디자인', '변화', '울트라', '기존', '진', '디자인', '모서리', ...","['둥근', '있습니다', '동일한', '있음']",[],"['되었으며', '와', '보입니다']","디자인, 변화, 울트라, 기존, 진, 디자인, 모서리, 변경, 카메라, 모듈, 디자...","둥근, 있습니다, 동일한, 있음",NaN,"되었으며, 와, 보입니다",1,"[둥글다, ,, 있다, ,, 동일하다, ,, 있다]",[],"[되어다, ,, 오다, ,, 보이다]"
1,공감아,2025-01-11T22:47:22+09:00,7만원 8만원에 물려있는게 대부분이라 7만원까지 혹여 올라도 물려있던분들은 원금 회...,5930,"['대부분', '혹', '던', '원금', '회복', '시', '다시', '풀', ...",[],[],"['물려있는게', '올라도', '물려있']","대부분, 혹, 던, 원금, 회복, 시, 다시, 풀, 매도, 다시",NaN,NaN,"물려있는게, 올라도, 물려있",0,[],[],"[물리다, ,, 오르다, ,, 물리다]"
2,곰돌이와콩이,2025-01-11T22:41:04+09:00,분할매수하기에 좋은 타점이지만.. 얘는 너무느리고 오래걸린다 얘를살바에 살게 너무많...,5930,"['분할', '타점', '얘', '얘', '살바', '살', '단점']","['좋은', '느리고', '많다는']","['너무', '오래', '너무']","['매수하기에', '걸린다']","분할, 타점, 얘, 얘, 살바, 살, 단점","좋은, 느리고, 많다는","너무, 오래, 너무","매수하기에, 걸린다",1,"[좋다, ,, 느리다, ,, 많다]","[너무, ,, 오래, ,, 너무]","[매수, 하다, ,, 걸리다]"
3,김브래,2025-01-11T21:31:55+09:00,존버하면 오르겠죠?,5930,['존버하'],[],[],['오르겠죠'],존버하,NaN,NaN,오르겠죠,0,[],[],[오르다]
4,김보리씨,2025-01-11T21:07:54+09:00,오를 겁니다. 제가 오늘 열심히 일하고 퇴근했습니다.,5930,"['제', '오늘', '일', '퇴근']",[],['열심히'],"['오를', '겁니다', '했습니다']","제, 오늘, 일, 퇴근",NaN,열심히,"오를, 겁니다, 했습니다",1,[],[열심히],"[오르다, ,, 겁니다, ,, 하다]"


In [31]:
positive_dict.head(10)

,점수
단어,
가까스로,0.0
가까이하다,1.0
가꾸러뜨리다,-1.0
가꾸러트리다,-1.0
가난,-2.0
가난뱅이,-2.0
가난살이,-2.0
가난살이하다,-2.0
가난설음,-2.0


In [32]:
result2 = pd.read_csv('community_token_final.csv', low_memory=False)

In [33]:
result2.head()

,닉네임,날짜,내용,종목,내용_명사,내용_형용사,내용_부사,내용_동사,내용_명사_str,내용_형용사_str,내용_부사_str,내용_동사_str
0,GROK,2025-01-11T22:59:18+09:00,- 디자인 변화: S25 울트라는 기존의 각진 디자인에서 둥근 모서리로 변경되었으며...,5930,"['디자인', '변화', '울트라', '기존', '진', '디자인', '모서리', ...","['둥근', '있습니다', '동일한', '있음']",[],"['되었으며', '와', '보입니다']","디자인, 변화, 울트라, 기존, 진, 디자인, 모서리, 변경, 카메라, 모듈, 디자...","둥근, 있습니다, 동일한, 있음",NaN,"되었으며, 와, 보입니다"
1,공감아,2025-01-11T22:47:22+09:00,7만원 8만원에 물려있는게 대부분이라 7만원까지 혹여 올라도 물려있던분들은 원금 회...,5930,"['대부분', '혹', '던', '원금', '회복', '시', '다시', '풀', ...",[],[],"['물려있는게', '올라도', '물려있']","대부분, 혹, 던, 원금, 회복, 시, 다시, 풀, 매도, 다시",NaN,NaN,"물려있는게, 올라도, 물려있"
2,곰돌이와콩이,2025-01-11T22:41:04+09:00,분할매수하기에 좋은 타점이지만.. 얘는 너무느리고 오래걸린다 얘를살바에 살게 너무많...,5930,"['분할', '타점', '얘', '얘', '살바', '살', '단점']","['좋은', '느리고', '많다는']","['너무', '오래', '너무']","['매수하기에', '걸린다']","분할, 타점, 얘, 얘, 살바, 살, 단점","좋은, 느리고, 많다는","너무, 오래, 너무","매수하기에, 걸린다"
3,김브래,2025-01-11T21:31:55+09:00,존버하면 오르겠죠?,5930,['존버하'],[],[],['오르겠죠'],존버하,NaN,NaN,오르겠죠
4,김보리씨,2025-01-11T21:07:54+09:00,오를 겁니다. 제가 오늘 열심히 일하고 퇴근했습니다.,5930,"['제', '오늘', '일', '퇴근']",[],['열심히'],"['오를', '겁니다', '했습니다']","제, 오늘, 일, 퇴근",NaN,열심히,"오를, 겁니다, 했습니다"


In [34]:
# 각 열에 대해 okt.morphs(stem=True) 적용
result2['형용사_어간'] = result2['내용_형용사_str'].apply(lambda x: okt.morphs(x, stem=True) if isinstance(x, str) and x != 'NaN' else [])
result2['부사_어간'] = result2['내용_부사_str'].apply(lambda x: okt.morphs(x, stem=True) if isinstance(x, str) and x != 'NaN' else [])
result2['동사_어간'] = result2['내용_동사_str'].apply(lambda x: okt.morphs(x, stem=True) if isinstance(x, str) and x != 'NaN' else [])

In [39]:
result2['형용사_어간'] = result2['형용사_어간'].apply(lambda x: ', '.join(x) if isinstance(x, list) else str(x))
result2['부사_어간'] = result2['부사_어간'].apply(lambda x: ', '.join(x) if isinstance(x, list) else str(x))
result2['동사_어간'] = result2['동사_어간'].apply(lambda x: ', '.join(x) if isinstance(x, list) else str(x))


In [ ]:
# 행별 총 긍정 점수를 계산할 파생열 추가
row_scores = [0] * len(result2)  # 초기화

for idx in range(len(result2)):  # 데이터프레임 행 반복
    row_score = 0  # 행별 총 점수 초기화

    for column in ['내용_명사_str', '형용사_어간', '부사_어간', '동사_어간']:
        word_list = result2.loc[idx, column]  # 각 열의 데이터 가져오기
        print(f"\n행 {idx}, 열 {column} 처리 중: {word_list}")
        
        if pd.isna(word_list) or word_list.strip() == "":  # Null 또는 빈 문자열 처리
            print(f"행 {idx}, 열 {column}은 결측값 또는 빈 문자열입니다. 건너뜁니다.")
            continue
        
        words = word_list.split(', ')  # 쉼표와 공백으로 구분
        for word in words:
            word = word.strip()  # 공백 제거
            if word in positive_dict.index:
                score = positive_dict.loc[word, '점수']
                row_score += score  # 점수 합산
                print(f"  단어: {word}, 점수: {score}, 누적 점수: {row_score}")
            else:
                print(f"  단어: {word}는 사전에 없음")

    row_scores[idx] = row_score  # 행별 총 점수 저장
    print(f"행 {idx}의 총 점수: {row_score}")

# 점수 변환: 음수는 -1, 0은 0, 양수는 1
try:
    result2['긍정점수'] = [1 if score > 0 else -1 if score < 0 else 0 for score in row_scores]

except Exception as e:
    print(f"점수 변환 중 에러 발생: {e}")
    print(f"row_scores 내용: {row_scores}")
    print(f"tmp 길이: {len(result2)}, row_scores 길이: {len(row_scores)}")

# 최종 점수 출력
print("\n행별 긍정점수:")
print(result2[['긍정점수']])

In [42]:
result2.head()

,닉네임,날짜,내용,종목,내용_명사,내용_형용사,내용_부사,내용_동사,내용_명사_str,내용_형용사_str,내용_부사_str,내용_동사_str,형용사_어간,부사_어간,동사_어간,긍정점수
0,GROK,2025-01-11T22:59:18+09:00,- 디자인 변화: S25 울트라는 기존의 각진 디자인에서 둥근 모서리로 변경되었으며...,5930,"['디자인', '변화', '울트라', '기존', '진', '디자인', '모서리', ...","['둥근', '있습니다', '동일한', '있음']",[],"['되었으며', '와', '보입니다']","디자인, 변화, 울트라, 기존, 진, 디자인, 모서리, 변경, 카메라, 모듈, 디자...","둥근, 있습니다, 동일한, 있음",NaN,"되었으며, 와, 보입니다","둥글다, ,, 있다, ,, 동일하다, ,, 있다",,"되어다, ,, 오다, ,, 보이다",1
1,공감아,2025-01-11T22:47:22+09:00,7만원 8만원에 물려있는게 대부분이라 7만원까지 혹여 올라도 물려있던분들은 원금 회...,5930,"['대부분', '혹', '던', '원금', '회복', '시', '다시', '풀', ...",[],[],"['물려있는게', '올라도', '물려있']","대부분, 혹, 던, 원금, 회복, 시, 다시, 풀, 매도, 다시",NaN,NaN,"물려있는게, 올라도, 물려있",,,"물리다, ,, 오르다, ,, 물리다",0
2,곰돌이와콩이,2025-01-11T22:41:04+09:00,분할매수하기에 좋은 타점이지만.. 얘는 너무느리고 오래걸린다 얘를살바에 살게 너무많...,5930,"['분할', '타점', '얘', '얘', '살바', '살', '단점']","['좋은', '느리고', '많다는']","['너무', '오래', '너무']","['매수하기에', '걸린다']","분할, 타점, 얘, 얘, 살바, 살, 단점","좋은, 느리고, 많다는","너무, 오래, 너무","매수하기에, 걸린다","좋다, ,, 느리다, ,, 많다","너무, ,, 오래, ,, 너무","매수, 하다, ,, 걸리다",1
3,김브래,2025-01-11T21:31:55+09:00,존버하면 오르겠죠?,5930,['존버하'],[],[],['오르겠죠'],존버하,NaN,NaN,오르겠죠,,,오르다,0
4,김보리씨,2025-01-11T21:07:54+09:00,오를 겁니다. 제가 오늘 열심히 일하고 퇴근했습니다.,5930,"['제', '오늘', '일', '퇴근']",[],['열심히'],"['오를', '겁니다', '했습니다']","제, 오늘, 일, 퇴근",NaN,열심히,"오를, 겁니다, 했습니다",,열심히,"오르다, ,, 겁니다, ,, 하다",1


In [43]:
result2['긍정점수'].value_counts()

긍정점수
 0    52095
-1    15583
 1    13394
Name: count, dtype: int64